# Query VizieR tables by coordinates

This notebook showcases, how we can use both `astroquery` and `pyVO` to query a catalogue in VizieR at different locations on the sky. 

## Import packages

In [1]:
from astroquery.vizier import Vizier
from astropy.coordinates import SkyCoord
from astropy.table import Table
import astropy.units as u
import pyvo

## Define the list of coordinates

In [2]:
ra = [120.689, 150.3684, 4.9516]
dec = [-23.4578, -65.794236, 35.5631]
coord_list = SkyCoord(ra=ra, dec=dec, unit=u.deg)

In [3]:
coord_list

<SkyCoord (ICRS): (ra, dec) in deg
    [(120.689 , -23.4578  ), (150.3684, -65.794236),
     (  4.9516,  35.5631  )]>

In [4]:
tab = Table([ra, dec], names=('ra_in', 'dec_in'))
tab.write('test2.vot', format='votable', overwrite=True)

In [5]:
tab

ra_in,dec_in
float64,float64
120.689,-23.4578
150.3684,-65.794236
4.9516,35.5631


## Using astroquery
In a first step we will need to find the identifier of the catalogue in VizieR. In this example here, we will want to query the 2MASS point source catalogue at different locations. 

In [6]:
catalog_list = Vizier.find_catalogs('2MASS PSC')

In [9]:
for k,v in catalog_list.items():
    print(k, v.description)

II/246 2MASS All-Sky Catalog of Point Sources (Cutri+ 2003)
II/281 2MASS 6X Point Source Working Database / Catalog (Cutri+ 2006)
J/ApJS/184/138 XID II: RASS/BSC-2MASS/PSC cross-association (Haakonsen+, 2009)


Now we can see that the 2MASS point source catalogue is known to VizieR as `II/246` and wen can use `query_region` to get all sources in the vicintity of our positions of interest. Note this can cause timeout errors if the list of coordinates is larger than 1000. 

In [10]:
result = Vizier.query_region(coord_list, radius=0.5 * u.arcmin, catalog='II/246')

In [11]:
result[0]

_q,RAJ2000,DEJ2000,_2MASS,Jmag,e_Jmag,Hmag,e_Hmag,Kmag,e_Kmag,Qflg,Rflg,Bflg,Cflg,Xflg,Aflg
,deg,deg,,mag,mag,mag,mag,mag,mag,,,,,,
int32,float64,float64,bytes17,float32,float32,float32,float32,float32,float32,bytes3,bytes3,bytes3,bytes3,uint8,uint8
1,120.685300,-23.459026,08024447-2327324,15.319,0.044,14.958,0.075,14.912,0.122,AAB,222,111,000,0,0
1,120.692687,-23.462269,08024624-2327441,16.032,0.082,15.740,0.176,15.372,0.195,ACC,222,111,000,0,0
1,120.692854,-23.457108,08024628-2327255,14.648,0.037,14.505,0.065,14.417,0.077,AAA,222,111,000,0,0
2,150.369660,-65.789169,10012871-6547210,14.800,0.036,14.514,0.054,14.408,0.092,AAA,222,111,000,0,0
2,150.356646,-65.798775,10012559-6547555,16.214,0.112,15.401,0.116,15.170,0.163,BBC,222,111,000,0,0
2,150.353369,-65.790985,10012480-6547275,15.351,0.056,14.590,0.055,14.459,0.089,AAA,222,111,000,0,0
2,150.364223,-65.795151,10012741-6547425,16.180,0.101,15.565,0.124,15.442,0.231,ABD,222,111,000,0,0
2,150.349593,-65.793053,10012390-6547349,16.083,0.091,15.836,0.158,15.175,--,ACU,220,110,000,0,0


## Using pyVO
This example shows how we can use the TAP module of pyVO to query VizieR around our list of coordinates. For this exercise we need the coordinates either in an `astropy.table.Table` object or saved in a VOTable. First we declare the TAP server that we want to query:

In [21]:
viz = pyvo.dal.TAPService('http://TAPVizieR.u-strasbg.fr/TAPVizieR/tap')

Next we define the ADQL query, which we want to run:

In [51]:
query = "SELECT tm.\"2MASS\", tm.RAJ2000, tm.DEJ2000, my_input.ra_in, my_input.dec_in " + \
        "FROM \"II/246/out\" as tm  " + \
        "JOIN TAP_UPLOAD.mytab as my_input " + \
        "ON 1=CONTAINS(POINT('ICRS', tm.RAJ2000, tm.DEJ2000),  " + \
        "              CIRCLE('ICRS', my_input.ra_in, my_input.dec_in, 0.5/60 )) "

Then we run the query. For large tables of coordinates, we might be interested in asynchronous queries but for some reason `run_async` does not work for me at the moment... 

In [52]:
results = viz.run_sync(query, uploads={'mytab': tab})

In [53]:
results

<Table length=10>
     2MASS        RAJ2000    DEJ2000    ra_in     dec_in  
                    deg        deg                        
    bytes17       float64    float64   float64   float64  
---------------- ---------- ---------- -------- ----------
08024624-2327441 120.692687 -23.462269  120.689   -23.4578
08024447-2327324   120.6853 -23.459026  120.689   -23.4578
08024628-2327255 120.692854 -23.457108  120.689   -23.4578
10012559-6547555 150.356646 -65.798775 150.3684 -65.794236
10012741-6547425 150.364223 -65.795151 150.3684 -65.794236
10013180-6547542  150.38251 -65.798416 150.3684 -65.794236
10012871-6547210  150.36966 -65.789169 150.3684 -65.794236
10012390-6547349 150.349593 -65.793053 150.3684 -65.794236
10012480-6547275 150.353369 -65.790985 150.3684 -65.794236
00194788+3533191   4.949531  35.555321   4.9516    35.5631